<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#PCA-of-correlation-matrix-(risks-x-canonical-variates)" data-toc-modified-id="PCA-of-correlation-matrix-(risks-x-canonical-variates)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>PCA of correlation matrix (risks x canonical variates)</a></span><ul class="toc-item"><li><span><a href="#Boostrap-of-split-half-analysis" data-toc-modified-id="Boostrap-of-split-half-analysis-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Boostrap of split-half analysis</a></span></li><li><span><a href="#Original-split-half-analysis" data-toc-modified-id="Original-split-half-analysis-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Original split-half analysis</a></span><ul class="toc-item"><li><span><a href="#Random-subset-1" data-toc-modified-id="Random-subset-1-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Random subset 1</a></span></li><li><span><a href="#Random-subset-2" data-toc-modified-id="Random-subset-2-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Random subset 2</a></span></li></ul></li><li><span><a href="#Bootstrapped-analysis-of-sex-specific-PCA" data-toc-modified-id="Bootstrapped-analysis-of-sex-specific-PCA-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Bootstrapped analysis of sex-specific PCA</a></span></li><li><span><a href="#PCA-on-males-only" data-toc-modified-id="PCA-on-males-only-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>PCA on males only</a></span></li><li><span><a href="#PCA-on-females-only" data-toc-modified-id="PCA-on-females-only-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>PCA on females only</a></span></li></ul></li><li><span><a href="#All-participant-PCA" data-toc-modified-id="All-participant-PCA-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>All participant PCA</a></span></li></ul></div>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from matplotlib.pyplot import figure
from scipy import stats
from tqdm import tqdm
import os

In [ ]:
data = pd.read_csv('HCDMNCOV_AD.csv')

In [ ]:
data.head()

In [ ]:
data.shape

**SUBGROUPING**

In [ ]:
#sex
id_m = np.where((data['Sex']==1))
id_f = np.where((data['Sex']==0))
#age
id_young = np.where(data['Age']<=55.0)
id_old = np.where(data['Age']>55.0)
#sex x age 
id_young_m = np.where((data['Age']<=55.0)&(data['Sex']==1))
id_young_f = np.where((data['Age']<=55.0)&(data['Sex']==0))
id_old_m = np.where((data['Age']>55.0)&(data['Sex']==1))
id_old_f = np.where((data['Age']>55.0)&(data['Sex']==0))
#education
id_low_ed = np.where(data['Education score']<=6.69)
id_high_ed = np.where(data['Education score']>6.69)
#all 
id_all = np.where(data['eid.1']!= None)

**GET FEATURES**

In [ ]:
def features (id):
    features = data.iloc[id]
    AD = features['mf']
    features_apoe = features[['e3/e3', 'e1/e3','e2/e2','e2/e3','e3/e4','e4/e4']]
    cca_modes = features[['1',
         '2',
         '3',
         '4',
         '5',
         '6',
         '7',
         '8',
         '9',
         '10',
         '11',
         '12',
         '13',
         '14',
         '15',
         '16',
         '17',
         '18',
         '19',
         '20',
         '21',
         '22',
         '23',
         '24',
         '25',
         '26',
         '27',
         '28',
         '29',
         '30',
         '31',
         '32',
         '33',
         '34',
         '35',
         '36',
         '37',
         '38',
         '39',
         '40',
         '41',
         '42',
         '43',
         '44',
         '45',
         '46',
         '47',
         '48',
         '49',
         '50']]
    risks = features[['Fluid intelligence score (R)',
              'Loneliness',
              'Lack of social support',
              'Age',
              'Sex',
              'Education score',
              'Age completed high school education',
              'Alcohol intake frequency',
              'Alcohol consumption on a typical drinking day',
              'Current tobacco smoking frequency',
              'Past tobacco smoking frequency',
              'Attend sports club or gym',
              'Attend pub or social club',
              'Attend religious group',
              'Attend adult education class',
              'Walking for pleasure',
              'Moderate exercises',
              'Strenuous sports',
              'Sleep duration',
              'Getting up in the morning',
              'Being a morning person',
              'Sleeplessness or insomnia',
              'Heart attack',
              'Angina',
              'Stroke',
              'Hypertension',
              'Diabetes diagnosed by a doctor',
              'Hearing difficulty with background noise',
              'Hearing aid user',
              'Glaucoma',
              'Cataract',
              'Macular degeneration',
              'Miserableness',
              'Fed-up feelings',
              'Mood swings',
              'Worrier / anxious feelings',
              'Nervous feelings',
              'Sensitivity / hurt feelings',
              "Tense / 'highly strung'",
              "Suffer from 'nerves'",
              'Worry too long after embarrassment',
              'Irritability',
              'Neuroticism score',
              'Happiness',
              'Feelings of nervousness or anxiety',
              'Frequency of friend / family visits',
              'Friendships satisfaction',
              'Family relationship satisfaction',
              'Number of full siblings',
              'Living in urban areas',
              'Average household income',
              'Paid employment',
              'Retirement',
              'Looking after home or family',
              'Unable to work due to sickness or disability',
              'Unemployment',
              'Doing unpaid or voluntary work',
              'Full or part-time student',
              'Number of vehicles in household',
              'Time spent watching TV',
              'Time spent using computer',
              'Number in household',
              'Physical environment score']]
    
    risks = risks.rename(columns={'Worrier / anxious feelings': 'Worrier_anxious',
                           'Fluid intelligence score (R)': 'Fluid intelligence score',
                           'Sensitivity / hurt feelings': 'Sensitivity_Hurt feelings',
                            "Tense / 'highly strung'":'Tense_Highly strung',
                           'Frequency of friend / family visits': 'Frequency of friend or family visits'})
    
    return AD, features_apoe, cca_modes, risks

In [ ]:
#generating features for all participants
AD, features_apoe, all_modes, risks = features(id_all)

In [ ]:
#z-scoring the risk factors
scaler = StandardScaler()
risks_z_scored = scaler.fit_transform(risks)

# PCA of correlation matrix (risks x canonical variates)

In [ ]:
#correlation between CCA modes and risk factors
def get_coefs(all_modes,risks, sex=False):
    
    scaler = StandardScaler()
    all_modes_zscored = scaler.fit_transform(all_modes)
    all_modes = pd.DataFrame(all_modes_zscored, columns=range(1,51))
    
    scaler = StandardScaler()
    risks_z_scored = scaler.fit_transform(risks)
    
    
    all_coefs = pd.DataFrame()
    for col in all_modes.columns:
        coef = []
        for i in range(0,len(risks_z_scored.T)):
            corr_matrix = np.corrcoef(all_modes[col],risks_z_scored.T[i])
            #print(corr_matrix)
            coef.append(corr_matrix[1][0])
        if sex == True:
            coef = np.array(coef).reshape(1,62)
        else:
            coef = np.array(coef).reshape(1,63)
        #index = np.array(index).flatten()
        #print(coef)
        #print(coef.shape)
        coef = pd.DataFrame(coef, columns = risks.columns)
        all_coefs = all_coefs.append(coef)
    all_coefs = all_coefs.reset_index(drop=True)
    all_coefs.index += 1
    return all_coefs

## Boostrap of split-half analysis

In [ ]:
for i in tqdm(range(0,1000)):
    
    #resample with replacment 37291 participants to create a bootstrap distribution
    rand_dist = resample(data, replace=True, random_state = i, n_samples = 37291)
    rand_dist = rand_dist.reset_index()
    
    #generating two random subset of equal size 
    rand1 = rand_dist.sample(frac=0.5,random_state=i)
    rand2 = rand_dist[~rand_dist.index.isin(rand1.index)]
    
    #get Pearson's correlations coefficients for both random subsets
    rand1_coefs = get_coefs(features(rand1.index)[2],features(rand1.index)[3])
    rand2_coefs = get_coefs(features(rand2.index)[2],features(rand2.index)[3])
    
    #z-scoring subset 1 
    scaler = StandardScaler()
    rand1_coefs = scaler.fit_transform(rand1_coefs)
    #PCA for subset 1 
    pca_1 = PCA(n_components=3)
    risks_transformed_1 = pca_1.fit_transform(rand1_coefs.T)
    
    #saving results
    if i ==0:
        rand1_cpm = np.array(pca_1.components_.T)
        rand1_risks = np.array(risks_transformed_1)
        rand1_exp_var = np.array(pca_1.explained_variance_ratio_)
    else:
        rand1_cpm = np.append(rand1_cpm, pca_1.components_.T, axis=1)
        rand1_risks = np.append(rand1_risks, risks_transformed_1, axis=1)
        rand1_exp_var = np.append(rand1_exp_var, pca_1.explained_variance_ratio_)
    
#     #test
#     print(rand1_cpm.shape)
#     print(rand1_risks.shape) 

    #z-scoring subset 2
    scaler = StandardScaler()
    rand2_coefs = scaler.fit_transform(rand2_coefs)
    #PCA for subset 2
    pca_2 = PCA(n_components=3)
    risks_transformed_2 = pca_2.fit_transform(rand2_coefs.T)

    #saving results
    if i ==0:
        rand2_cpm = np.array(pca_2.components_.T)
        rand2_risks = np.array(risks_transformed_2)
        rand2_exp_var = np.array(pca_2.explained_variance_ratio_)
    else:
        rand2_cpm = np.append(rand2_cpm, pca_2.components_.T, axis=1)
        rand2_risks = np.append(rand2_risks, risks_transformed_2, axis=1)
        rand2_exp_var = np.append(rand2_exp_var, pca_2.explained_variance_ratio_)
    
#     #test
#     print(rand2_cpm.shape)
#     print(rand2_risks.shape)  

In [ ]:
cols = []
for i in range(1,26):
    cols.append(f'HC_{i}')
for i in range(1,26):
    cols.append(f'DN_{i}')

In [ ]:
%matplotlib inline
figure(figsize=(10, 10), dpi=80)
corr_mat = np.corrcoef(rand1_cpm[:,::3], -1*rand2_cpm[:,::3])
sns.heatmap(corr_mat[50:,:50], cmap=plt.cm.RdBu_r, center=0, vmin=-1, vmax=+1, yticklabels=cols,xticklabels=cols)
plt.title('Pearson\'s correlations of the weights for component 1 across bootstrapped iterations for random subsets 1 and 2')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
plt.savefig('component_1_pearson_bootstrapped_weights.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
%matplotlib inline
figure(figsize=(10, 10), dpi=80)
corr_mat = np.corrcoef(rand1_cpm[:,1::3], -1*rand2_cpm[:,1::3])
sns.heatmap(corr_mat[50:,:50], cmap=plt.cm.RdBu_r, center=0, vmin=-1, vmax=+1, yticklabels=cols,xticklabels=cols)
plt.title('Pearson\'s correlations of the weights for component 2 across bootstrapped iterations for random subsets 1 and 2')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
plt.savefig('component_2_pearson_bootstrapped_weights.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
%matplotlib inline
figure(figsize=(10, 10), dpi=80)
corr_mat = np.corrcoef(rand1_cpm[:,2::3], -1*rand2_cpm[:,2::3])
sns.heatmap(corr_mat[50:,:50], cmap=plt.cm.RdBu_r, center=0, vmin=-1, vmax=+1, yticklabels=cols,xticklabels=cols)
plt.title('Pearson\'s correlations of the weights for component 3 across bootstrapped iterations for random subsets 1 and 2')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
plt.savefig('component_3_pearson_bootstrapped_weights.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
%matplotlib inline
figure(figsize=(10, 10), dpi=80)
corr_mat = np.corrcoef(rand1_risks[:,::3], -1*rand2_risks[:,::3])
sns.heatmap(corr_mat[63:,:63], cmap=plt.cm.RdBu_r, center=0, vmin=-1, vmax=+1, yticklabels=risks.columns,xticklabels=risks.columns)
plt.title('Pearson\'s correlations of the projections on component 1 across bootstrapped iterations for random subsets 1 and 2')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
plt.savefig('component_1_pearson_bootstrapped_projections.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
%matplotlib inline
figure(figsize=(10, 10), dpi=80)
corr_mat = np.corrcoef(rand1_risks[:,1::3], -1*rand2_risks[:,1::3])
sns.heatmap(corr_mat[63:,:63], cmap=plt.cm.RdBu_r, center=0, vmin=-1, vmax=+1, yticklabels=risks.columns,xticklabels=risks.columns)
plt.title('Pearson\'s correlations of the projections on component 2 across bootstrapped iterations for random subsets 1 and 2')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
plt.savefig('component_2_pearson_bootstrapped_projections.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
%matplotlib inline
figure(figsize=(10, 10), dpi=80)
corr_mat = np.corrcoef(rand1_risks[:,2::3], -1*rand2_risks[:,2::3])
sns.heatmap(corr_mat[63:,:63], cmap=plt.cm.RdBu_r, center=0, vmin=-1, vmax=+1, yticklabels=risks.columns,xticklabels=risks.columns)
plt.title('Pearson\'s correlations of the projections on component 3 across bootstrapped iterations for random subsets 1 and 2')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
plt.savefig('component_3_pearson_bootstrapped_projections.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
np.savetxt("rand1_cpm.csv", rand1_cpm, delimiter=",")
np.savetxt("rand2_cpm.csv", rand2_cpm, delimiter=",")

np.savetxt("rand1_risks.csv", rand1_risks, delimiter=",")
np.savetxt("rand2_risks.csv", rand2_risks, delimiter=",")

np.savetxt("rand1_exp_var.csv", rand1_exp_var, delimiter=",")
np.savetxt("rand2_exp_var.csv", rand2_exp_var, delimiter=",")

## Original split-half analysis

In [ ]:
rand1 = data.sample(frac=0.5,random_state=42)
rand2 = data[~data.index.isin(rand1.index)]

In [ ]:
rand1_coefs = get_coefs(features(rand1.index)[2],features(rand1.index)[3])
rand2_coefs = get_coefs(features(rand2.index)[2],features(rand2.index)[3])

In [ ]:
#z-scoring
scaler = StandardScaler()
rand1_coefs = scaler.fit_transform(rand1_coefs)

scaler = StandardScaler()
rand2_coefs = scaler.fit_transform(rand2_coefs)

### Random subset 1

In [ ]:
pca = PCA(n_components=3)
risks_transformed = pca.fit_transform(rand1_coefs.T)
pca.explained_variance_ratio_

In [ ]:
%matplotlib inline
figure(figsize=(5, 5), dpi=80)
sns.heatmap(pca.components_[:,0:10], cmap='PiYG', square = True, center=0, yticklabels=['PC1','PC2','PC3'], xticklabels=range(1,11), cbar_kws={"shrink": 0.2}, vmin = -0.5, vmax = 0.5)
plt.title('Eigenvectors for HC patterns')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
#plt.savefig('pca_eigenvetcors_rand1_HC.png', dpi=200, bbox_inches='tight')

In [ ]:
%matplotlib inline
figure(figsize=(5, 5), dpi=80)
sns.heatmap(pca.components_[:,25:35], cmap='PiYG', square = True, center=0, yticklabels=['PC1','PC2','PC3'], xticklabels=range(1,11), cbar_kws={"shrink": 0.2}, vmin = -0.5, vmax = 0.5)
plt.title('Eigenvectors for DN patterns')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
plt.savefig('pca_eigenvetcors_rand1_DN.png', dpi=200, bbox_inches='tight')

In [ ]:
%matplotlib inline
figure(figsize=(4, 10), dpi=80)
sns.heatmap(risks_transformed, cmap=plt.cm.RdBu_r, center=0, xticklabels=['PC1','PC2','PC3'], yticklabels=risks.drop('Sex',axis=1).columns,vmax=5, vmin=-5)
plt.title('Projections of the Pearson\'s correlations onto the PCs for random subset 1')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
plt.savefig('rand1_pca.png', dpi=200, bbox_inches='tight')
plt.show()

### Random subset 2

In [ ]:
pca = PCA(n_components=3)
risks_transformed = pca.fit_transform(rand2_coefs.T)
pca.explained_variance_ratio_

In [ ]:
%matplotlib inline
figure(figsize=(5, 10), dpi=80)
sns.heatmap(pca.components_[:,0:10], cmap='PiYG', square = True, center=0, yticklabels=['PC1','PC2','PC3'], xticklabels=range(1,11), cbar_kws={"shrink": 0.4}, vmin = -0.5, vmax = 0.5)
plt.title('Eigenvectors for HC patterns')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
plt.savefig('pca_eigenvetcors_rand2_HC.png', dpi=200, bbox_inches='tight')

In [ ]:
%matplotlib inline
figure(figsize=(5, 5), dpi=80)
sns.heatmap(pca.components_[:,25:35], cmap='PiYG', square = True, center=0, yticklabels=['PC1','PC2','PC3'], xticklabels=range(1,11), cbar_kws={"shrink": 0.2}, vmin = -0.5, vmax = 0.5)
plt.title('Eigenvectors for DN patterns')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
plt.savefig('pca_eigenvetcors_rand2_DN.png', dpi=200, bbox_inches='tight')

In [ ]:
%matplotlib inline
figure(figsize=(4, 10), dpi=80)
sns.heatmap(risks_transformed, cmap=plt.cm.RdBu_r, center=0, xticklabels=['PC1','PC2','PC3'], yticklabels=risks.drop('Sex',axis=1).columns, cbar_kws={"shrink": 0.4}, vmax=5, vmin=-5)
plt.title('Projections of the Pearson\'s correlations onto the PCs for for random subset 2')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
plt.savefig('rand2_pca.png', dpi=200, bbox_inches='tight')
plt.show()

## Bootstrapped analysis of sex-specific PCA 

In [ ]:
#generating features for males participants
AD_m, features_apoe_m, all_modes_m, risks_m = features(id_m)

In [ ]:
#generating features for females participants
AD_f, features_apoe_f, all_modes_f, risks_f = features(id_f)

In [ ]:
for i in tqdm(range(0,1000)):
    
    AD_m, features_apoe_m, all_modes_m, risks_m = features(id_m)
    AD_f, features_apoe_f, all_modes_f, risks_f = features(id_f)
    
    all_modes_m, risks_m = resample(all_modes_m, risks_m.drop('Sex',axis=1), replace=True, random_state = i, n_samples = 17561)
    all_modes_f, risks_f = resample(all_modes_f, risks_f.drop('Sex',axis=1), replace=True, random_state = i, n_samples = 19730)
    
    all_coefs_m = get_coefs(all_modes_m,risks_m, sex=True)
    all_coefs_f = get_coefs(all_modes_f,risks_f, sex=True)
    
    #z-scoring males
    scaler = StandardScaler()
    z_all_coefs_m = scaler.fit_transform(all_coefs_m)
    
#     print(z_all_coefs_m.shape)
    
    #PCA for males
    pca_m = PCA(n_components=3)
    risks_transformed_m = pca_m.fit_transform(z_all_coefs_m.T)
    
    #saving results
    if i ==0:
        males_cpm = np.array(pca_m.components_.T)
        males_risks = np.array(risks_transformed_m)
        males_exp_var = np.array(pca_m.explained_variance_ratio_)
    else:
        males_cpm = np.append(males_cpm, pca_m.components_.T, axis=1)
        males_risks = np.append(males_risks, risks_transformed_m, axis=1)
        males_exp_var = np.append(males_exp_var, pca_m.explained_variance_ratio_)
    
#     #test
#     print(males_cpm.shape)
#     print(males_risks.shape) 

    #z-scoring females
    scaler = StandardScaler()
    z_all_coefs_f = scaler.fit_transform(all_coefs_f)
    
#     print(z_all_coefs_f.shape)
    
    #PCA for females
    pca_f = PCA(n_components=3)
    risks_transformed_f = pca_f.fit_transform(z_all_coefs_f.T)

    #saving results
    if i ==0:
        females_cpm = np.array(pca_f.components_.T)
        females_risks = np.array(risks_transformed_f)
        females_exp_var = np.array(pca_f.explained_variance_ratio_)
    else:
        females_cpm = np.append(females_cpm, pca_f.components_.T, axis=1)
        females_risks = np.append(females_risks, risks_transformed_f, axis=1)
        females_exp_var = np.append(females_exp_var, pca_f.explained_variance_ratio_)
    
#     #test
#     print(females_cpm.shape)
#     print(females_risks.shape)  

In [ ]:
%matplotlib inline
figure(figsize=(10, 10), dpi=80)
corr_mat = np.corrcoef(males_cpm[:,::3], females_cpm[:,::3])
sns.heatmap(corr_mat[50:,:50], cmap=plt.cm.RdBu_r, center=0, vmin=-1, vmax=+1, yticklabels=cols,xticklabels=cols)
plt.title('Pearson\'s correlations of the weights for component 1 across bootstrapped iterations for PCA ran on males and females')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
plt.savefig('component_1_pearson_bootstrapped_weights_sex.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
%matplotlib inline
figure(figsize=(10, 10), dpi=80)
corr_mat = np.corrcoef(males_cpm[:,1::3], females_cpm[:,1::3])
sns.heatmap(corr_mat[50:,:50], cmap=plt.cm.RdBu_r, center=0, vmin=-1, vmax=+1, yticklabels=cols,xticklabels=cols)
plt.title('Pearson\'s correlations of the weights for component 2 across bootstrapped iterations for PCA ran on males and females')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
plt.savefig('component_2_pearson_bootstrapped_weights_sex.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
%matplotlib inline
figure(figsize=(10, 10), dpi=80)
corr_mat = np.corrcoef(males_cpm[:,2::3], females_cpm[:,2::3])
sns.heatmap(corr_mat[50:,:50], cmap=plt.cm.RdBu_r, center=0, vmin=-1, vmax=+1, yticklabels=cols,xticklabels=cols)
plt.title('Pearson\'s correlations of the weights for component 3 across bootstrapped iterations for PCA ran on males and females')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
plt.savefig('component_3_pearson_bootstrapped_weights_sex.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
%matplotlib inline
figure(figsize=(10, 10), dpi=80)
corr_mat = np.corrcoef(males_risks[:,::3], females_risks[:,::3])
sns.heatmap(corr_mat[62:,:62], cmap=plt.cm.RdBu_r, center=0, vmin=-1, vmax=+1, yticklabels=risks_m.columns,xticklabels=risks_m.columns)
plt.title('Pearson\'s correlations of the projections on component 1 across bootstrapped iterations for PCA ran on males and females')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
plt.savefig('component_1_pearson_bootstrapped_projections_sex.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
%matplotlib inline
figure(figsize=(10, 10), dpi=80)
corr_mat = np.corrcoef(males_risks[:,1::3], females_risks[:,1::3])
sns.heatmap(corr_mat[62:,:62], cmap=plt.cm.RdBu_r, center=0, vmin=-1, vmax=+1, yticklabels=risks_m.columns,xticklabels=risks_m.columns)
plt.title('Pearson\'s correlations of the projections on component 2 across bootstrapped iterations for PCA ran on males and females')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
plt.savefig('component_2_pearson_bootstrapped_projections_sex.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
%matplotlib inline
figure(figsize=(10, 10), dpi=80)
corr_mat = np.corrcoef(males_risks[:,2::3], females_risks[:,2::3])
sns.heatmap(corr_mat[62:,:62], cmap=plt.cm.RdBu_r, center=0, vmin=-1, vmax=+1, yticklabels=risks_m.columns,xticklabels=risks_m.columns)
plt.title('Pearson\'s correlations of the projections on component 3 across bootstrapped iterations for PCA ran on males and females')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
plt.savefig('component_3_pearson_bootstrapped_projections_sex.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
np.savetxt("females_cpm.csv", females_cpm, delimiter=",")
np.savetxt("males_cpm.csv", males_cpm, delimiter=",")

np.savetxt("females_risks.csv", females_risks, delimiter=",")
np.savetxt("males_risks.csv", males_risks, delimiter=",")

np.savetxt("females_exp_var.csv", females_exp_var, delimiter=",")
np.savetxt("males_exp_var.csv", males_exp_var, delimiter=",")

## PCA on males only 

In [ ]:
all_coefs_m = get_coefs(features(id_m)[2],features(id_m)[3].drop('Sex',axis=1))

In [ ]:
scaler = StandardScaler()
all_coefs_m = scaler.fit_transform(all_coefs_m)

In [ ]:
pca = PCA(n_components=3)
risks_transformed = pca.fit_transform(all_coefs_m.T)
pca.explained_variance_ratio_

In [ ]:
%matplotlib inline
figure(figsize=(5, 5), dpi=80)
sns.heatmap(pca.components_[:,0:10], cmap='PiYG', square = True, center=0, yticklabels=['PC1','PC2','PC3'], xticklabels=range(1,11), cbar_kws={"shrink": 0.2}, vmin = -0.5, vmax = 0.5)
plt.title('Eigenvectors for HC patterns')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
plt.savefig('pca_eigenvetcors_males_HC.png', dpi=200, bbox_inches='tight')

In [ ]:
%matplotlib inline
figure(figsize=(5, 5), dpi=80)
sns.heatmap(pca.components_[:,25:35], cmap='PiYG', square = True, center=0, yticklabels=['PC1','PC2','PC3'], xticklabels=range(1,11), cbar_kws={"shrink": 0.2}, vmin = -0.5, vmax = 0.5)
plt.title('Eigenvectors for DN patterns')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
plt.savefig('pca_eigenvetcors_males_DN.png', dpi=200, bbox_inches='tight')

In [ ]:
%matplotlib inline
figure(figsize=(4, 10), dpi=80)
sns.heatmap(risks_transformed, cmap=plt.cm.RdBu_r, center=0, xticklabels=['PC1','PC2','PC3'], yticklabels=risks.drop('Sex',axis=1).columns, cbar_kws={"shrink": 0.4}, vmax=5, vmin=-5)
plt.title('Projections of the Pearson\'s correlations onto the PCs in males')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
plt.savefig('males_pca.png', dpi=200, bbox_inches='tight')

## PCA on females only

In [ ]:
all_coefs_f = get_coefs(features(id_f)[2],features(id_f)[3].drop('Sex',axis=1))

In [ ]:
scaler = StandardScaler()
all_coefs_f = scaler.fit_transform(all_coefs_f)

In [ ]:
pca = PCA(n_components=3)
risks_transformed = pca.fit_transform(all_coefs_f.T)
pca.explained_variance_ratio_

In [ ]:
%matplotlib inline
figure(figsize=(5, 5), dpi=80)
sns.heatmap(pca.components_[:,0:10], cmap='PiYG', square = True, center=0, yticklabels=['PC1','PC2','PC3'], xticklabels=range(1,11), cbar_kws={"shrink": 0.2}, vmin = -0.5, vmax = 0.5)
plt.title('Eigenvectors for HC patterns')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
plt.savefig('pca_eigenvetcors_females_HC.png', dpi=200, bbox_inches='tight')

In [ ]:
%matplotlib inline
figure(figsize=(5, 5), dpi=80)
sns.heatmap(pca.components_[:,25:35], cmap='PiYG', square = True, center=0, yticklabels=['PC1','PC2','PC3'], xticklabels=range(1,11), cbar_kws={"shrink": 0.2}, vmin = -0.5, vmax = 0.5)
plt.title('Eigenvectors for DN patterns')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
plt.savefig('pca_eigenvetcors_females_DN.png', dpi=200, bbox_inches='tight')

In [ ]:
# XXX of each mode to each latent risk factor
%matplotlib inline
figure(figsize=(4, 10), dpi=80)
sns.heatmap(risks_transformed, cmap=plt.cm.RdBu_r, center=0, xticklabels=['PC1','PC2','PC3'], yticklabels=risks.drop('Sex',axis=1).columns, cbar_kws={"shrink": 0.4}, vmax=5, vmin=-5)
plt.title('Projections of the Pearson\'s correlations onto the PCs in females')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
plt.savefig('pca_females.png', dpi=200, bbox_inches='tight')

# All participant PCA

In [ ]:
all_coefs = get_coefs(features(id_all)[2],features(id_all)[3])

scaler = StandardScaler()
all_coefs_zscored = scaler.fit_transform(all_coefs)

pca = PCA(n_components=3)
risks_transformed = pca.fit_transform(all_coefs_zscored.T)

In [ ]:
#correlation of each mode to each latent risk factor
%matplotlib inline
figure(figsize=(5, 5), dpi=80)
sns.heatmap(pca.components_[:,0:10], cmap='PiYG', square = True, center=0, yticklabels=['PC1','PC2','PC3'], xticklabels=range(1,11), cbar_kws={"shrink": 0.2}, vmin = -0.5, vmax = 0.5)
plt.title('Eigenvectors for HC patterns')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
#plt.savefig('pca_eigenvetcors_all_HC.png', dpi=200, bbox_inches='tight')

In [ ]:
%matplotlib inline
figure(figsize=(5, 5), dpi=80)
sns.heatmap(pca.components_[:,25:35], cmap='PiYG', square = True, center=0, yticklabels=['PC1','PC2','PC3'], xticklabels=range(1,11), cbar_kws={"shrink": 0.2}, vmin = -0.5, vmax = 0.5)
plt.title('Eigenvectors for DN patterns')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
#plt.savefig('pca_eigenvetcors_all_DN.png', dpi=200, bbox_inches='tight')

In [ ]:
# XXX of each mode to each latent risk factor
%matplotlib inline
figure(figsize=(4, 10), dpi=80)
sns.heatmap(risks_transformed, cmap=plt.cm.RdBu_r, center=0, xticklabels=['PC1','PC2','PC3'], yticklabels=risks.columns, cbar_kws={"shrink": 0.2}, vmax=5, vmin=-5)
plt.title('Projections of the Pearson\'s correlations onto the PCs')
#plot1_path = 'CCA_plots/all/CCA_APOE_AD/DMN_APOE_interactions_AD.png'
#plt.tight_layout()
plt.subplots_adjust(top=1)
plt.savefig('pca_projections.png', dpi=200, bbox_inches='tight')